<a href="https://colab.research.google.com/github/maryamteimouri/Thesis-SentimentAnalysis/blob/validation-assessment/Ann_Agreement_Round2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Agreement

In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from collections import defaultdict
from itertools import combinations

file_path1 = 'Kian.jsonl'
file_path2 = 'Annina.jsonl'

In [ ]:
# Define separate label sets for each task
label_mapping_speaker = ['Speaker 1', 'Speaker 2', 'Interviewee', 'Interviewer', 'Instructor']
label_mapping_emotion = ['Joy', 'Sadness', 'Anger', 'Fear', 'Surprise', 'Disgust', 'Trust', 'Anticipation']

# Group IDs (modify as needed)
group1_ids = {1, 2, 3, 4}
group2_ids = {9, 11, 15}
group3_ids = {5, 6, 7, 8, 10, 12, 13, 14, 16}
group_definitions = {'Group 1': group1_ids, 'Group 2': group2_ids, 'Group 3': group3_ids}

## Multi-labeled

In [ ]:
# Function to load JSONL data
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Function to aggregate annotations for each group
def aggregate_annotations_multilabel(spans, group_ids):
    aggregated_text = ""
    aggregated_annotations = []
    offset = 0

    for doc in spans:
        if doc['id'] in group_ids:
            text = doc['text']
            annotations = [(start + offset, end + offset, label) for start, end, label in doc['annotations']]
            aggregated_text += text
            aggregated_annotations.extend(annotations)
            offset += len(text)  # Adjust offset for next document in group

    return aggregated_text, aggregated_annotations

# Create one-hot encoding for multi-label annotations
def one_hot_encode_annotations(annotations, labels):
    encoding = []
    label_to_idx = {label: i for i, label in enumerate(labels)}
    for label_set in annotations:
        vec = [0] * len(labels)
        for label in label_set:
            if label in label_to_idx:
                vec[label_to_idx[label]] = 1
        encoding.append(vec)
    return encoding

# Jaccard similarity calculation
def jaccard_similarity_for_labels(annotator1, annotator2):
    """Calculate Jaccard Similarity for overlapping labels."""
    similarities = []
    for labels1, labels2 in zip(annotator1, annotator2):
        if labels1 or labels2:  # Only calculate for labeled positions
            intersection = len(set(labels1) & set(labels2))
            union = len(set(labels1) | set(labels2))
            similarity = intersection / union if union > 0 else 0
            similarities.append(similarity)
    return np.mean(similarities)

# Pad vectors to ensure consistent length for comparison
def pad_to_match_length(vec1, vec2, pad_value=0):
    """Pad two lists to match the length of the longer list."""
    max_len = max(len(vec1), len(vec2))
    return vec1 + [pad_value] * (max_len - len(vec1)), vec2 + [pad_value] * (max_len - len(vec2))

# Calculate multi-label agreement for grouped data for each task separately
def calculate_multi_label_agreement_for_tasks(spans_annotator1, spans_annotator2, labels_speaker, labels_emotion, group_definitions):
    results = []
    for group_name, group_ids in group_definitions.items():
        # Aggregate annotations for each group
        text_1, annotations_1 = aggregate_annotations_multilabel(spans_annotator1, group_ids)
        text_2, annotations_2 = aggregate_annotations_multilabel(spans_annotator2, group_ids)

        # Convert aggregated annotations to list of labels per position for each annotator
        annotator1_labels = [[] for _ in range(len(text_1))]
        annotator2_labels = [[] for _ in range(len(text_2))]

        for start, end, label in annotations_1:
            for i in range(start, end):
                annotator1_labels[i].append(label)

        for start, end, label in annotations_2:
            for i in range(start, end):
                annotator2_labels[i].append(label)

        # Speaker Identification Task
        speaker_jaccard = jaccard_similarity_for_labels(
            [[label for label in labels if label in labels_speaker] for labels in annotator1_labels],
            [[label for label in labels if label in labels_speaker] for labels in annotator2_labels]
        )

        speaker_encoded1 = one_hot_encode_annotations(
            [[label for label in labels if label in labels_speaker] for labels in annotator1_labels],
            labels_speaker
        )
        speaker_encoded2 = one_hot_encode_annotations(
            [[label for label in labels if label in labels_speaker] for labels in annotator2_labels],
            labels_speaker
        )

        speaker_kappas = []
        for label_idx in range(len(labels_speaker)):
            label_vec1 = [pos[label_idx] for pos in speaker_encoded1]
            label_vec2 = [pos[label_idx] for pos in speaker_encoded2]
            padded_vec1, padded_vec2 = pad_to_match_length(label_vec1, label_vec2)
            if any(padded_vec1) or any(padded_vec2):
                speaker_kappas.append(cohen_kappa_score(padded_vec1, padded_vec2))
        speaker_kappa = np.mean(speaker_kappas) if speaker_kappas else None

        # Emotion Detection Task
        emotion_jaccard = jaccard_similarity_for_labels(
            [[label for label in labels if label in labels_emotion] for labels in annotator1_labels],
            [[label for label in labels if label in labels_emotion] for labels in annotator2_labels]
        )

        emotion_encoded1 = one_hot_encode_annotations(
            [[label for label in labels if label in labels_emotion] for labels in annotator1_labels],
            labels_emotion
        )
        emotion_encoded2 = one_hot_encode_annotations(
            [[label for label in labels if label in labels_emotion] for labels in annotator2_labels],
            labels_emotion
        )

        emotion_kappas = []
        for label_idx in range(len(labels_emotion)):
            label_vec1 = [pos[label_idx] for pos in emotion_encoded1]
            label_vec2 = [pos[label_idx] for pos in emotion_encoded2]
            padded_vec1, padded_vec2 = pad_to_match_length(label_vec1, label_vec2)
            if any(padded_vec1) or any(padded_vec2):
                emotion_kappas.append(cohen_kappa_score(padded_vec1, padded_vec2))
        emotion_kappa = np.mean(emotion_kappas) if emotion_kappas else None

        # Append results for this group for each task
        results.append({
            'group': group_name,
            'speaker_jaccard': speaker_jaccard,
            'speaker_kappa': speaker_kappa,
            'emotion_jaccard': emotion_jaccard,
            'emotion_kappa': emotion_kappa
        })

    return pd.DataFrame(results)

# Load your files and run
data_annotator1 = load_jsonl(file_path1)
data_annotator2 = load_jsonl(file_path2)

# Prepare spans for both annotators
def get_spans_for_jaccard(data):
    spans = []
    for doc in data:
        text = doc['text']
        doc_annotations = [(start, end, label) for start, end, label in doc['label']]
        spans.append({'id': doc['id'], 'text': text, 'annotations': doc_annotations})
    return spans

spans1 = get_spans_for_jaccard(data_annotator1)
spans2 = get_spans_for_jaccard(data_annotator2)

# Run agreement calculation for each group and task
agreement_results_grouped_tasks = calculate_multi_label_agreement_for_tasks(
    spans1, spans2, label_mapping_speaker, label_mapping_emotion, group_definitions
)

# Display the results
print(file_path1,file_path2)
print(agreement_results_grouped_tasks)


FileNotFoundError: [Errno 2] No such file or directory: 'Kian.jsonl'

## Labeled Only

In [ ]:
# Function to load JSONL data
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Function to aggregate annotations for each group
def aggregate_annotations_multilabel(spans, group_ids):
    aggregated_text = ""
    aggregated_annotations = []
    offset = 0

    for doc in spans:
        if doc['id'] in group_ids:
            text = doc['text']
            annotations = [(start + offset, end + offset, label) for start, end, label in doc['annotations']]
            aggregated_text += text
            aggregated_annotations.extend(annotations)
            offset += len(text)  # Adjust offset for next document in group

    return aggregated_text, aggregated_annotations

# Create one-hot encoding for multi-label annotations
def one_hot_encode_annotations(annotations, labels):
    encoding = []
    label_to_idx = {label: i for i, label in enumerate(labels)}
    for label_set in annotations:
        vec = [0] * len(labels)
        for label in label_set:
            if label in label_to_idx:
                vec[label_to_idx[label]] = 1
        encoding.append(vec)
    return encoding

# Jaccard similarity calculation
def jaccard_similarity_for_labels(annotator1, annotator2):
    """Calculate Jaccard Similarity for overlapping labels."""
    similarities = []
    for labels1, labels2 in zip(annotator1, annotator2):
        if labels1 and labels2:  # Only calculate for labeled positions
            intersection = len(set(labels1) & set(labels2))
            union = len(set(labels1) | set(labels2))
            similarity = intersection / union if union > 0 else 0
            similarities.append(similarity)
    return np.mean(similarities)

# Calculate multi-label agreement for grouped data for each task separately
def calculate_multi_label_agreement_for_tasks(spans_annotator1, spans_annotator2, labels_speaker, labels_emotion, group_definitions):
    results = []
    for group_name, group_ids in group_definitions.items():
        # Aggregate annotations for each group
        text_1, annotations_1 = aggregate_annotations_multilabel(spans_annotator1, group_ids)
        text_2, annotations_2 = aggregate_annotations_multilabel(spans_annotator2, group_ids)

        # Convert aggregated annotations to list of labels per position for each annotator
        annotator1_labels = [[] for _ in range(len(text_1))]
        annotator2_labels = [[] for _ in range(len(text_2))]

        for start, end, label in annotations_1:
            for i in range(start, end):
                annotator1_labels[i].append(label)

        for start, end, label in annotations_2:
            for i in range(start, end):
                annotator2_labels[i].append(label)

        # Speaker Identification Task
        speaker_jaccard = jaccard_similarity_for_labels(
            [[label for label in labels if label in labels_speaker] for labels in annotator1_labels],
            [[label for label in labels if label in labels_speaker] for labels in annotator2_labels]
        )

        # Emotion Detection Task
        emotion_jaccard = jaccard_similarity_for_labels(
            [[label for label in labels if label in labels_emotion] for labels in annotator1_labels],
            [[label for label in labels if label in labels_emotion] for labels in annotator2_labels]
        )

        # Append results for this group for each task
        results.append({
            'group': group_name,
            'speaker_jaccard': speaker_jaccard,
            'emotion_jaccard': emotion_jaccard
        })

    return pd.DataFrame(results)

# Load your files and run
data_annotator1 = load_jsonl(file_path1)
data_annotator2 = load_jsonl(file_path2)

# Prepare spans for both annotators
def get_spans_for_jaccard(data):
    spans = []
    for doc in data:
        text = doc['text']
        doc_annotations = [(start, end, label) for start, end, label in doc['label']]
        spans.append({'id': doc['id'], 'text': text, 'annotations': doc_annotations})
    return spans

spans1 = get_spans_for_jaccard(data_annotator1)
spans2 = get_spans_for_jaccard(data_annotator2)

# Run agreement calculation for each group and task
agreement_results_grouped_tasks = calculate_multi_label_agreement_for_tasks(
    spans1, spans2, label_mapping_speaker, label_mapping_emotion, group_definitions
)

# Display the results
print(file_path1,file_path2)
print(agreement_results_grouped_tasks)


## Sentiment Only

In [ ]:
# Define positive and negative emotions based on the user's instruction
positive_emotions = {'Joy', 'Surprise', 'Trust', 'Anticipation'}
negative_emotions = {'Sadness', 'Anger', 'Fear', 'Disgust'}

# Function to classify labels as positive or negative
def classify_sentiment_labels(labels, positive_emotions, negative_emotions):
    """Classify a list of labels as 'positive', 'negative', or [] if neither."""
    sentiment_labels = []
    for label_set in labels:
        if any(label in positive_emotions for label in label_set):
            sentiment_labels.append("positive")
        elif any(label in negative_emotions for label in label_set):
            sentiment_labels.append("negative")
        else:
            sentiment_labels.append([])  # Empty if no sentiment category applies
    return sentiment_labels

# Function to calculate sentiment agreement
def calculate_sentiment_agreement(spans_annotator1, spans_annotator2, positive_emotions, negative_emotions, group_definitions):
    results = []
    for group_name, group_ids in group_definitions.items():
        # Aggregate annotations for each group
        text_1, annotations_1 = aggregate_annotations_multilabel(spans_annotator1, group_ids)
        text_2, annotations_2 = aggregate_annotations_multilabel(spans_annotator2, group_ids)

        # Convert aggregated annotations to list of labels per position for each annotator
        annotator1_labels = [[] for _ in range(len(text_1))]
        annotator2_labels = [[] for _ in range(len(text_2))]

        for start, end, label in annotations_1:
            for i in range(start, end):
                annotator1_labels[i].append(label)

        for start, end, label in annotations_2:
            for i in range(start, end):
                annotator2_labels[i].append(label)

        # Classify labels by sentiment (positive or negative)
        annotator1_sentiment = classify_sentiment_labels(annotator1_labels, positive_emotions, negative_emotions)
        annotator2_sentiment = classify_sentiment_labels(annotator2_labels, positive_emotions, negative_emotions)

        # Filter to only keep positions where both annotators have classified a sentiment
        sentiment_jaccard = jaccard_similarity_for_labels(
            [[sentiment] for sentiment in annotator1_sentiment if sentiment],
            [[sentiment] for sentiment in annotator2_sentiment if sentiment]
        )

        # Append results for this group
        results.append({
            'group': group_name,
            'sentiment_jaccard': sentiment_jaccard
        })

    return pd.DataFrame(results)

# Run sentiment agreement calculation
sentiment_agreement_results = calculate_sentiment_agreement(
    spans1, spans2, positive_emotions, negative_emotions, group_definitions
)

# Display the results
print(file_path1,file_path2)
print(sentiment_agreement_results)


## Speaker switch

In [ ]:
def aggregate_annotations_multilabel(spans, group_ids):
    """Aggregate annotations from spans for a specific group."""
    text = ''
    annotations = []
    for span in spans:
        if span['id'] in group_ids:
            text += span['text']  # Concatenate text for the group
            annotations.extend(span['annotations'])  # Extend annotations list
    return text, annotations

def identify_speaker_switches(labels):
    """Identify positions where a speaker switch occurs in the annotation labels."""
    switch_points = [0] * len(labels)  # Initialize a list to mark switches
    for i in range(1, len(labels)):
        if labels[i] != labels[i - 1]:  # A switch occurs if the label changes from previous
            switch_points[i] = 1
    return switch_points

def get_spans_for_kappa(data):
    """Convert JSONL data into spans format expected by calculation functions."""
    spans = []
    for doc in data:
        text = doc['text']
        doc_annotations = [(start, end, label) for start, end, label in doc['label']]
        spans.append({'id': doc.get('id'), 'text': text, 'annotations': doc_annotations})
    return spans

In [ ]:
# Adjust group definitions based on the user's request
group_definitions = {
    'Group1': {1, 2, 3, 4},
    'Group2': {9, 11, 15},
    'Group3': {5, 6, 7, 8, 10, 12, 13, 14, 16},
}

# List of labels to filter
valid_labels = {'Speaker 1', 'Speaker 2', 'Interviewee', 'Interviewer', 'Instructor'}

def calculate_speaker_switch_agreement_filtered(spans_annotator1, spans_annotator2, group_definitions, valid_labels):
    results = []
    for group_name, group_ids in group_definitions.items():
        # Aggregate annotations for each group
        text_1, annotations_1 = aggregate_annotations_multilabel(spans_annotator1, group_ids)
        text_2, annotations_2 = aggregate_annotations_multilabel(spans_annotator2, group_ids)

        # Convert aggregated annotations to list of labels per position for each annotator
        annotator1_labels = [[] for _ in range(len(text_1))]
        annotator2_labels = [[] for _ in range(len(text_2))]

        for start, end, label in annotations_1:
            if label in valid_labels:  # Only include valid labels
                for i in range(start, end):
                    annotator1_labels[i].append(label)

        for start, end, label in annotations_2:
            if label in valid_labels:  # Only include valid labels
                for i in range(start, end):
                    annotator2_labels[i].append(label)

        # Simplify to primary speaker label per position, ignore multi-label
        annotator1_primary_labels = [labels[0] if labels else None for labels in annotator1_labels]
        annotator2_primary_labels = [labels[0] if labels else None for labels in annotator2_labels]

        # Identify switch points for each annotator
        annotator1_switches = identify_speaker_switches(annotator1_primary_labels)
        annotator2_switches = identify_speaker_switches(annotator2_primary_labels)

        # Calculate Cohen's Kappa for switch points
        switch_kappa = cohen_kappa_score(annotator1_switches, annotator2_switches)

        # Append results for this group
        results.append({
            'group': group_name,
            'switch_kappa': switch_kappa
        })

    return pd.DataFrame(results)

# Calculate Kappa for the filtered data
speaker_switch_agreement_results_filtered = calculate_speaker_switch_agreement_filtered(spans1, spans2, group_definitions, valid_labels)

# Display the results
print(file_path1,file_path2)
print(speaker_switch_agreement_results_filtered)


# GPTs and Human

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from collections import defaultdict
from itertools import combinations
import re

In [2]:
from transformers import AutoTokenizer

# Initialize the XLM-R tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [65]:
file_path1 = 'all.jsonl'
file_path2 = 'admin.jsonl'
group_ids_index = 1

In [66]:
# Define separate label sets for each task
label_mapping_speaker = ['Speaker 1', 'Speaker 2'] #, 'Speaker 1', 'Interviewer' , 'Instructor'
label_mapping_emotion = ['Joy', 'Sadness', 'Anger', 'Fear', 'Surprise', 'Disgust', 'Trust', 'Anticipation']

label_emotion = ['Joy', 'sadness', 'anger', 'Fear', 'Surprise', 'disgust', 'Trust', 'anticipation']
label_speaker = ['S1', 'S2'] #'interviewee', 'interviewer', 'Instructor', 'Interviewee','interviwer'

# Group IDs (modify as needed)
group1_ids = {1, 2, 3, 4}
group2_ids = {9, 11, 15}
group3_ids = {5, 6, 7, 8, 10, 12, 13, 14, 16}
group_definitions = {'Group 1': group1_ids, 'Group 2': group2_ids, 'Group 3': group3_ids}

In [67]:
def identify_unwanted_spans(text):
    # Find spans for unwanted patterns (filenames and timestamps)
    unwanted_spans = []
    pattern = r'\b\w+\.txt\b|\d{2}:\d{2}:\d{2},\d{3}(\.\d+)? --> \d{2}:\d{2}:\d{2},\d{3}(\.\d+)?'
    for match in re.finditer(pattern, text):
        unwanted_spans.append((match.start(), match.end()))
    return unwanted_spans

def filter_tokens_based_on_spans(tokens, labels, unwanted_spans):
    # Initialize lists to hold the filtered tokens and labels
    filtered_tokens = []
    filtered_labels = []

    # Go through each token and its offset
    for token, label, (start, end) in zip(tokens['input_ids'], labels, tokens['offset_mapping']):
        # Check if the token's position overlaps with any unwanted span
        if not any(start >= u_start and end <= u_end for u_start, u_end in unwanted_spans):
            filtered_tokens.append(token)
            filtered_labels.append(label)

    return filtered_tokens, filtered_labels

def add_segments(data, annotator):
    """
    Adds segments to each document based on numeric or timestamp markers.
    """
    segmented_data = []
    for doc in data:
        text = doc['text']
        annotations = doc.get('annotations', [])
        # Split text into segments using markers (e.g., numeric or timestamp)
        #segments = re.split(r'(\n\d+\n)', text)
        pattern = r'(\b\w+\.txt\b)?(\n\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3})'

        if annotator  == 'ann2':
            pattern = r'(\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3})'

        # Use `finditer` instead of `re.split` for better control
        matches = list(re.finditer(pattern, text))

        # Collect segments
        segments = []
        for i, match in enumerate(matches):
            start = match.start()
            end = match.end()
            next_start = matches[i + 1].start() if i + 1 < len(matches) else len(text)

            # Include everything from the start of the match to the start of the next match
            segment_text = text[start:next_start].strip()
            segment_offset = start  # Use the segment's start point as the offset

            # Adjust annotations for this segment
            segment_annotations = [
                (start - segment_offset, end - segment_offset, label)
                for start, end, label in annotations
                if start >= segment_offset and end <= segment_offset + len(segment_text)
            ]

            # Add the segmented data
            segmented_data.append({
                "id": doc['id'],
                "text": segment_text,
                "annotations": segment_annotations
            })
    return segmented_data

## Speakers and Switches

In [62]:
def identify_speaker_switches(labels):
    """Identify positions where a speaker switch occurs in the annotation labels."""
    switch_points = [0] * len(labels)  # Initialize a list to mark switches
    for i in range(1, len(labels)):
        if labels[i] != labels[i - 1]:  # A switch occurs if the label changes from previous
            switch_points[i] = 1
    return switch_points

In [70]:
# Function to load JSONL data
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

# Prepare spans for both annotators
def get_spans_for_jaccard(data):
    spans = []
    for doc in data:
        text = doc['text']
        doc_annotations = [(start, end, label) for start, end, label in doc['label']]
        if 'id' in doc:
          spans.append({'id': doc['id'], 'text': text, 'annotations': doc_annotations})
        else:
          spans.append({'id': -1, 'text': text, 'annotations': doc_annotations})

    spans.sort(key=lambda x: x['id'])
    return spans

def tokenize_and_adjust_annotations(text, annotations, max_length=512, overlap=50):

    # Step 1: Identify unwanted spans in the raw text
    unwanted_spans = identify_unwanted_spans(text)

    tokens = tokenizer(
        text,
        return_offsets_mapping=True,
        add_special_tokens=False,
        max_length=max_length,
        truncation=True
    )
    input_ids = tokens['input_ids']
    offset_mapping = tokens['offset_mapping']

    # Initialize the token annotations
    final_token_annotations = [[] for _ in range(len(offset_mapping))]

    # Map annotations to tokens
    for start, end, label in annotations:
        for token_idx, (token_start, token_end) in enumerate(offset_mapping):
            if token_start < end and token_end > start:  # Check for overlap
                final_token_annotations[token_idx].append(label)

    # Step 2: Filter tokens and annotations based on unwanted spans
    input_ids, final_token_annotations = filter_tokens_based_on_spans(tokens, final_token_annotations, unwanted_spans)

    return input_ids, final_token_annotations

def aggregate_annotations_multilabel_tokenized(doc):
    token_ids = []
    aggregated_annotations = []
    offset = 0

    text = doc['text']
    annotations = doc['annotations']

    # Tokenize and align annotations for the document
    tokenized_text, token_annotations = tokenize_and_adjust_annotations(text,
                                                                        annotations)
    token_ids.extend(tokenized_text)
    aggregated_annotations.extend(token_annotations)

    return token_ids, aggregated_annotations

# Create one-hot encoding for multi-label annotations
def one_hot_encode_annotations(annotations, labels):
    encoding = []
    label_to_idx = {label: i for i, label in enumerate(labels)}
    for label_set in annotations:
        vec = [0] * len(labels)
        for label in label_set:
            if label in label_to_idx:
                vec[label_to_idx[label]] = 1
        encoding.append(vec)
    return encoding

# Jaccard similarity calculation
def jaccard_similarity_for_labels(annotator1, annotator2):
    """Calculate Jaccard Similarity for overlapping labels."""
    similarities = []
    for labels1, labels2 in zip(annotator1, annotator2):
        if labels1 and labels2:  # Only calculate for labeled positions
            intersection = len(set(labels1) & set(labels2))
            union = len(set(labels1) | set(labels2))
            similarity = intersection / union if union > 0 else 0
            similarities.append(similarity)
    return np.mean(similarities)

# Pad vectors to ensure consistent length for comparison
def pad_to_match_length(vec1, vec2, pad_value=0):
    """Pad two lists to match the length of the longer list."""
    max_len = max(len(vec1), len(vec2))
    return vec1 + [pad_value] * (max_len - len(vec1)), vec2 + [pad_value] * (max_len - len(vec2))

def map_annotations_directly(annotations, annotator_labels):
    """
    Maps annotations directly to each token position assuming alignment by index.
    """
    for i, label in enumerate(annotations):
        if i < len(annotator_labels):  # Ensure within bounds
            annotator_labels[i].extend(label)

def calculate_multi_label_agreement_for_tasks(index, spans_annotator1,
                                              spans_annotator2, labels_speaker,
                                              group_definitions):
    annotator1_labels_final = []
    annotator2_labels_final = []
    group_name, group_ids = list(group_definitions.items())[index]
    annotator1_segments = add_segments(spans_annotator1, 'ann1')
    annotator2_segments = add_segments(spans_annotator2, 'ann2')

    for segment1, segment2 in zip(annotator1_segments, annotator2_segments):

        if segment1['id'] not in group_ids or segment2['id'] not in group_ids:
          continue
        # Tokenize and aggregate annotations for each group
        token_ids_1, annotations_1 = aggregate_annotations_multilabel_tokenized(segment1)
        token_ids_2, annotations_2 = aggregate_annotations_multilabel_tokenized(segment2)

        # Initialize lists to hold annotations for each token position
        annotator1_labels = [[] for _ in range(len(token_ids_1))]
        annotator2_labels = [[] for _ in range(len(token_ids_2))]

        # Map annotations directly to tokens by position
        map_annotations_directly(annotations_1, annotator1_labels)
        map_annotations_directly(annotations_2, annotator2_labels)

        annotator1_labels_final.extend([[label for label in labels if label in label_speaker] for labels in annotator1_labels])
        annotator2_labels_final.extend([[label for label in labels if label in labels_speaker] for labels in annotator2_labels])

    results = []
    # Speaker Identification Task

    speaker_encoded1 = one_hot_encode_annotations(
        annotator1_labels_final, label_speaker
    )
    speaker_encoded2 = one_hot_encode_annotations(
        annotator2_labels_final, labels_speaker
    )

    speaker_jaccard = jaccard_similarity_for_labels(
        speaker_encoded1, speaker_encoded2
    )

    speaker_kappas = []
    for label_idx in range(len(labels_speaker)):
        label_vec1 = [pos[label_idx] for pos in speaker_encoded1]
        label_vec2 = [pos[label_idx] for pos in speaker_encoded2]
        padded_vec1, padded_vec2 = pad_to_match_length(label_vec1, label_vec2)
        if any(padded_vec1) or any(padded_vec2):
            speaker_kappas.append(cohen_kappa_score(padded_vec1, padded_vec2))
    speaker_kappa = np.mean(speaker_kappas) if speaker_kappas else None

    # Identify switch points for each annotator
    annotator1_switches = identify_speaker_switches(annotator1_labels_final)
    annotator2_switches = identify_speaker_switches(annotator2_labels_final)

    # Calculate Cohen's Kappa for switch points
    min_length = min(len(annotator1_switches), len(annotator2_switches))
    switch_kappa = cohen_kappa_score(annotator1_switches[:min_length],
                                     annotator2_switches[:min_length])

    # Append results for this group for each task
    results.append({
        'group': group_name,
        'speaker_jaccard': speaker_jaccard,
        'speaker_kappa': speaker_kappa,
        'switch_kappa': switch_kappa
    })

    return pd.DataFrame(results)


In [71]:
# Load your files and run
data_annotator1 = load_jsonl(file_path1)
data_annotator2 = load_jsonl(file_path2)

spans1 = get_spans_for_jaccard(data_annotator1)
spans2 = get_spans_for_jaccard(data_annotator2)

# Run agreement calculation for each group and task
agreement_results_grouped_tasks = calculate_multi_label_agreement_for_tasks(
    group_ids_index, spans1, spans2, label_mapping_speaker, group_definitions
)

# Display the results
print(file_path1,file_path2)
print(agreement_results_grouped_tasks)

all.jsonl admin.jsonl
     group  speaker_jaccard  speaker_kappa  switch_kappa
0  Group 2         0.836402       0.005796      0.012516


## Sentiment

In [34]:
# Define positive and negative emotions based on the user's instruction
positive_emotions = {'Joy', 'Surprise', 'Trust', 'Anticipation', 'anticipation'}
negative_emotions = {'Sadness', 'Anger', 'Fear', 'Disgust', 'sadness', 'anger', 'Fear', 'disgust'}

# Function to classify labels as positive or negative
def classify_sentiment_labels(labels, positive_emotions, negative_emotions):
    """Classify a list of labels as 'positive', 'negative', or [] if neither."""
    sentiment_labels = []
    for label_set in labels:
        if any(label in positive_emotions for label in label_set):
            sentiment_labels.append("positive")
        elif any(label in negative_emotions for label in label_set):
            sentiment_labels.append("negative")
        else:
            sentiment_labels.append([])  # Empty if no sentiment category applies
    return sentiment_labels

In [47]:
# Function to load JSONL data
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def tokenize_and_adjust_annotations(text, annotations, max_length=512, overlap=50):
    # Step 1: Identify unwanted spans in the raw text
    unwanted_spans = identify_unwanted_spans(text)

    # Split the text into chunks that fit max_length with overlap
    text_chunks = []
    start = 0
    while start < len(text):
        end = min(start + max_length, len(text))
        text_chunks.append(text[start:end])
        start += max_length - overlap  # Slide with overlap

    # Initialize lists for final input_ids and annotations
    final_input_ids = []
    final_token_annotations = []

    # Process each text chunk
    for idx, chunk in enumerate(text_chunks):
        # Tokenize the chunk and get the offset mapping
        tokens = tokenizer(chunk, return_offsets_mapping=True,
                           add_special_tokens=False, max_length=max_length,
                           truncation=True)
        input_ids = tokens['input_ids']
        offset_mapping = tokens['offset_mapping']

        # Function to map annotations to tokens in a single chunk
        def map_annotations_to_tokens(chunk_offsets, annotations, chunk_start):
            chunk_annotations = [[] for _ in range(len(chunk_offsets))]
            for start, end, label in annotations:
                for token_idx, (token_start, token_end) in enumerate(chunk_offsets):
                    # Adjust start and end based on chunk start position
                    adj_token_start = token_start + chunk_start
                    adj_token_end = token_end + chunk_start
                    if adj_token_start < end and adj_token_end > start:  # Check for overlap
                        chunk_annotations[token_idx].append(label)
            return chunk_annotations

        # Map annotations to this specific chunk
        chunk_annotations = map_annotations_to_tokens(offset_mapping, annotations,
                                                      idx * (max_length - overlap))

        # Adjust overlap after the first chunk
        if idx > 0:
            input_ids = input_ids[overlap:]
            chunk_annotations = chunk_annotations[overlap:]

        # Step 2: Filter tokens and annotations based on unwanted spans
        input_ids, chunk_annotations = filter_tokens_based_on_spans(tokens, chunk_annotations, unwanted_spans)

        # Extend the final lists
        final_input_ids.extend(input_ids)
        final_token_annotations.extend(chunk_annotations)

    return final_input_ids, final_token_annotations

def aggregate_annotations_multilabel_tokenized(spans, group_ids):
    token_ids = []
    aggregated_annotations = []
    offset = 0

    for doc in spans:
        if doc['id'] not in group_ids:
            continue
        text = doc['text']
        annotations = doc['annotations']

        # Tokenize and align annotations for the document
        tokenized_text, token_annotations = tokenize_and_adjust_annotations(text,
                                                                            annotations)
        token_ids.extend(tokenized_text)
        aggregated_annotations.extend(token_annotations)

    return token_ids, aggregated_annotations

# Create one-hot encoding for multi-label annotations
def one_hot_encode_annotations(annotations, labels):
    encoding = []
    label_to_idx = {label: i for i, label in enumerate(labels)}
    for label_set in annotations:
        vec = [0] * len(labels)
        for label in label_set:
            if label in label_to_idx:
                vec[label_to_idx[label]] = 1
        encoding.append(vec)
    return encoding

# Jaccard similarity calculation
def jaccard_similarity_for_labels(annotator1, annotator2):
    """Calculate Jaccard Similarity for overlapping labels."""
    similarities = []
    for labels1, labels2 in zip(annotator1, annotator2):
        if labels1 or labels2:  # Only calculate for labeled positions
            intersection = len(set(labels1) & set(labels2))
            union = len(set(labels1) | set(labels2))
            similarity = intersection / union if union > 0 else 0
            similarities.append(similarity)
    return np.mean(similarities)

# Pad vectors to ensure consistent length for comparison
def pad_to_match_length(vec1, vec2, pad_value=0):
    """Pad two lists to match the length of the longer list."""
    max_len = max(len(vec1), len(vec2))
    return vec1 + [pad_value] * (max_len - len(vec1)), vec2 + [pad_value] * (max_len - len(vec2))

def map_annotations_directly(annotations, annotator_labels):
    """
    Maps annotations directly to each token position assuming alignment by index.
    """
    for i, label in enumerate(annotations):
        if i < len(annotator_labels):  # Ensure within bounds
            annotator_labels[i].extend(label)

def calculate_multi_label_agreement_for_tasks(index, spans_annotator1,
                                              spans_annotator2, labels_emotion,
                                               group_definitions):
    group_name, group_ids = list(group_definitions.items())[index]

    # Tokenize and aggregate annotations for each group
    token_ids_1, annotations_1 = aggregate_annotations_multilabel_tokenized(spans_annotator1, group_ids)
    token_ids_2, annotations_2 = aggregate_annotations_multilabel_tokenized(spans_annotator2, group_ids)

    # Initialize lists to hold annotations for each token position
    annotator1_labels = [[] for _ in range(len(token_ids_1))]
    annotator2_labels = [[] for _ in range(len(token_ids_2))]

    # Map annotations directly to tokens by position
    map_annotations_directly(annotations_1, annotator1_labels)
    map_annotations_directly(annotations_2, annotator2_labels)

    results = []

    # Classify labels by sentiment (positive or negative)
    annotator1_sentiment = classify_sentiment_labels(annotator1_labels, positive_emotions, negative_emotions)
    annotator2_sentiment = classify_sentiment_labels(annotator2_labels, positive_emotions, negative_emotions)

    # Filter to only keep positions where both annotators have classified a sentiment
    sentiment_jaccard = jaccard_similarity_for_labels(
        [[sentiment] for sentiment in annotator1_sentiment if sentiment],
        [[sentiment] for sentiment in annotator2_sentiment if sentiment]
    )

    # Emotion Detection Task
    emotion_jaccard = jaccard_similarity_for_labels(
        [[label for label in labels if label in label_emotion] for labels in annotator1_labels],
        [[label for label in labels if label in labels_emotion] for labels in annotator2_labels]
    )

    emotion_encoded1 = one_hot_encode_annotations(
        [[label for label in labels if label in label_emotion] for labels in annotator1_labels],
        label_emotion
    )
    emotion_encoded2 = one_hot_encode_annotations(
        [[label for label in labels if label in labels_emotion] for labels in annotator2_labels],
        labels_emotion
    )

    emotion_kappas = []
    for label_idx in range(len(labels_emotion)):
        label_vec1 = [pos[label_idx] for pos in emotion_encoded1]
        label_vec2 = [pos[label_idx] for pos in emotion_encoded2]
        padded_vec1, padded_vec2 = pad_to_match_length(label_vec1, label_vec2)
        if any(padded_vec1) or any(padded_vec2):
            emotion_kappas.append(cohen_kappa_score(padded_vec1, padded_vec2))
    emotion_kappa = np.mean(emotion_kappas) if emotion_kappas else None

    # Append results for this group for each task
    results.append({
        'group': group_name,
        'emotion_jaccard': emotion_jaccard,
        'emotion_kappa': emotion_kappa,
        'sentiment_jaccard': sentiment_jaccard
    })

    return pd.DataFrame(results)

# Prepare spans for both annotators
def get_spans_for_jaccard(data):
    spans = []
    for doc in data:
        text = doc['text']
        doc_annotations = [(start, end, label) for start, end, label in doc['label']]
        if 'id' in doc:
          spans.append({'id': doc['id'], 'text': text, 'annotations': doc_annotations})
        else:
          spans.append({'id': -1, 'text': text, 'annotations': doc_annotations})

    spans.sort(key=lambda x: x['id'])
    return spans


In [50]:
# Load your files and run
data_annotator1 = load_jsonl(file_path1)
data_annotator2 = load_jsonl(file_path2)

spans1 = get_spans_for_jaccard(data_annotator1)
spans2 = get_spans_for_jaccard(data_annotator2)

# Run agreement calculation for each group and task
agreement_results_grouped_tasks = calculate_multi_label_agreement_for_tasks(
    group_ids_index, spans1, spans2, label_mapping_emotion, group_definitions
)

# Display the results
print(file_path1,file_path2)
print(agreement_results_grouped_tasks)

all.jsonl admin.jsonl
     group  emotion_jaccard  emotion_kappa  sentiment_jaccard
0  Group 3          0.03926       0.010759            0.57415
